In [ ]:
# !conda install -y -n nbenv1 -c conda-forge regionmask cartopy pygeos
#!conda install -y -n nbenv1 nc-time-axis

In [1]:
# import cf, cfplot as cfp
import numpy as np
import scipy as sp
import sys
import os
from netCDF4 import Dataset
import netCDF4 as nc
%matplotlib inline
import matplotlib
import xarray as xr
import matplotlib.pyplot as plt
import warnings

import matplotlib.pyplot as plt
import matplotlib.path as mpath
# Quick plot to show the results
from cartopy import config
import cartopy.crs as ccrs
import cartopy
import cartopy.feature as cfeature
import scipy.ndimage as ndimage0
# from mpl_toolkits.basemap import Basemap
# from mpl_toolkits.basemap import addcyclic
from cartopy.util import add_cyclic_point
import matplotlib.pylab as pl
from scipy import stats
import regionmask
import glob
import dask
from general_functions import *
dask.config.set(**{'array.slicing.split_large_chunks': True})

In [2]:
seas='MAM'
reg='IPCC'
nyear=1
rebasing=False

if (seas == "DJF"):
    sind=1
else:
    sind=0

print(sind)
def rebase(dataset):
    # ds = xr.open_dataset('intermediate_files/stable/gmst/tas_gmst_bq777.nc') #,engine='zarr')
    dates=dataset[0,].dropna(dim='year').year  
    print(dates)
    datasets = []
    test=dataset
    for i in np.arange(0,6):
        tmp=test[i,:].dropna(dim='year')
        tmp['year']=dates
#         print(tmp.year)
        datasets.append(tmp)
    dsnew = xr.concat(datasets, dim='cases')
    # print(dsnew)
    return(dsnew)
def preprocess(ds):
    print(ds.encoding['source'])
    return(ds)

def seasaverage_3(var,seas):
    season={'DJF':0,'JFM':1,'FMA':2,'MAM':3,'AMJ':4,'MJJ':5,'JJA':6,'JAS':7,'ASO':8,'SON':9,'OND':10,'NDJ':11,'ann':None}
    seasindex=season[seas]
    if (seas == 'ann'):
        var_seas=var.groupby('time.year').mean('time')
    else:
        if (var.ndim == 4):
            var_seas=var.rolling(min_periods=3, center=True, time=3).mean()[:,seasindex::12,:,:]
            var_seas=var_seas.groupby('time.year').mean('time') #rename({"time":"year"})
        elif (var.ndim == 3): 
            var_seas=var.rolling(min_periods=3, center=True, time=3).mean()[seasindex::12,:,:]
            var_seas=var_seas.groupby('time.year').mean('time')
        print(var_seas)
    return(var_seas)

0


In [3]:
scenario='ssp370'
factor={'tas':1,'pr':86400,'txx':1}
gmstssp370={}
regtemp_ssp={}
gmst2={}
test={}
hist={}
base_region={}
piCbase_region={}
base_mask={}

zipgmststab=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_*.nc'))
zipgmst=xr.open_mfdataset(zipgmststab,combine='nested',concat_dim='cases',preprocess=preprocess)['tas']
zipgmst=np.nanmean(rebase(zipgmst),axis=1)
# print(zipgmst)

def sortbygmst(input):
    input = [x for _,x in sorted(zip(zipgmst,input))]
    return(input)

def calc_regions(var,lat1=None,lat2=None,lon1=None,lon2=None,units='None',mask='None'):
    fileshist=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/historical/'+var+'/'+var+'_r*.nc'))
    if (mask=='None'):
        hist=seasaverage_3(xr.open_mfdataset(fileshist,combine='nested',concat_dim='cases')[var],seas)*factor[var] # gmst timeseries for all hist runs --> baseperiod
    else:
        hist=mask_land(seasaverage_3(xr.open_mfdataset(fileshist,combine='nested',concat_dim='cases')[var],seas)*factor[var]).transpose('cases','year','lat','lon')
    print(hist)
    lat=hist['lat']
    lon=hist['lon']
    base=hist[:,0:50,:,:].mean('cases').mean('year')
    print(np.nanmax(base))
    if (lat1==None):
        base_mask=regionmask.defined_regions.ar6.land.mask_3D(base)
        hist_mask=regionmask.defined_regions.ar6.land.mask_3D(hist)
        print(base_mask[16].sel(lat=slice(50,60)).values)
        print(base_mask[16].sel(lat=slice(50,60)).values)
    else:
        base=lonflip(mask_land(base)).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2))
        base_mask=1.
        print(base_mask)


        
    # filespic=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/piControl/'+var+'/'+var+'_*_r*.nc'))
    # piC=seasaverage_3(xr.open_mfdataset(filespic,combine='nested',concat_dim='time')[var],seas)*factor[var] # gmst timeseries for all hist runs --> baseperiod
    # base_piC=piC.mean('year')
    # piCbase_mask=regionmask.defined_regions.ar6.land.mask_3D(base_piC)
    # weights = np.cos(np.deg2rad(base_piC.lat))
    # piCbase_region = base_piC.weighted(piCbase_mask * weights).mean(dim=("lat", "lon")).values

    piCbase_region = np.nan
    if (lat1==None):
        weights = np.cos(np.deg2rad(base.lat))
    else:
         weights = np.cos(np.deg2rad(base.lat)).sel(lat=slice(lat1,lat2))
    print(base.shape)
    base_region = base.weighted(base_mask*weights).mean(dim=("lat", "lon")).values
    if (units != '%'):
        ts_hist_regional = hist.weighted(hist_mask * weights).mean(dim=("lat", "lon"))-base_region
        print('no units specified')
    else:
        ts_hist_regional = 100*(hist.weighted(hist_mask * weights).mean(dim=("lat", "lon"))-base_region)/base_region
        print('units %')
    # base_region2d=np.repeat(base_region[np.newaxis,:],526, axis=0)
    # base_region3d=np.repeat(base_region2d[:,:,np.newaxis],6, axis=2)
    
    files=sortbygmst(sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/'+var+'/'+var+'_?????.nc')))
    tasfiles=xr.open_mfdataset(files,combine='nested',concat_dim='cases',preprocess=preprocess)
    if (mask=='None'):
        tas=seasaverage_3(tasfiles[var],seas)*factor[var]
    else:
        tas=mask_land(seasaverage_3(tasfiles[var],seas)*factor[var]).transpose('cases','year','lat','lon')
    print(np.nanmax(tas))
    if (var == 'txx'):
        tas=tas.rename({'latitude':'lat','longitude':'lon'})
    if (rebasing==True):
        tas=rebase(tas)
    transientfiles=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/'+scenario+'/'+var+'/'+var+'_r*.nc'))
    transienttas=xr.open_mfdataset(transientfiles,combine='nested',concat_dim='cases')
    if (mask=='None'):
        tas_ssp370=seasaverage_3(transienttas[var],seas)*factor[var]
    else:
        tas_ssp370=mask_land(seasaverage_3(transienttas[var],seas)*factor[var]).transpose('cases','year','lat','lon')
    if (lat1==None):
        mask_full = regionmask.defined_regions.ar6.land.mask_3D(tas)
    else:
        tas=lonflip(mask_land(tas)).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2))
        tas.to_netcdf(path='test.nc', mode='w')
        print(tas.lon)
        mask_full = 1.
    weights = np.cos(np.deg2rad(tas.lat))
    print(np.nanmax(base_region))
    if (units != '%'):
        ts_airtemps_regional = tas.weighted(mask_full * weights).mean(dim=("lat", "lon"))-base_region
        print('no units specified')
    else:
        ts_airtemps_regional = 100*(tas.weighted(mask_full * weights).mean(dim=("lat", "lon"))-base_region)/base_region
        print('units %')
    print(np.nanmax(ts_airtemps_regional))
# ts_airtemps_regional[:,:,:].plot.line(x='year',col="region", col_wrap=4)
    if (lat1==None):
        mask_transient = regionmask.defined_regions.ar6.land.mask_3D(tas_ssp370)
    else:
        tas_ssp370=lonflip(mask_land(tas_ssp370)).sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2))
        mask_transient = 1.
    weights2 = np.cos(np.deg2rad(tas_ssp370.lat))
    if (units != '%'):
        transient_airtemps_regional = tas_ssp370.weighted(mask_transient * weights2).mean(dim=("lat", "lon"))-base_region
        print('no units specified')
    else:
        print('units %')
        transient_airtemps_regional = 100*(tas_ssp370.weighted(mask_transient * weights2).mean(dim=("lat", "lon"))-base_region)/base_region
    # fig, axarr = plt.subplots(nrows=15, ncols=4, figsize=(10, 25), constrained_layout=True) # figsize width then height
# axlist = axarr.flatten()
    test=ts_airtemps_regional
    year=tas.year
# for i in np.arange(0,58):
#     axlist[i].plot(year,np.moveaxis(test[:,:,i],0,1))
# axlist[58].axis('off')
# axlist[59].axis('off')
# calculate stable gmst 

    gmststab=sortbygmst(sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_*.nc')))
    gmst=xr.open_mfdataset(gmststab,combine='nested',concat_dim='cases',preprocess=preprocess)['tas']
    if (rebasing == True):
        gmst=rebase(gmst)
    gmstbase=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/historical/gmst/tas_gmst_*.nc'))
    gmstref=xr.open_mfdataset(gmstbase,combine='nested',concat_dim='cases')['tas']
    base=gmstref[:,0:50].mean('cases').mean('year')
    gmst2=gmst-base

    # calculate transient_gmst 

    gmstssp=sorted(glob.glob('/gws/nopw/j04/realproj/users/adittus/intermediate_files/ssp370/gmst/tas_gmst_*.nc'))
    gmstsspt=xr.open_mfdataset(gmstssp,combine='nested',concat_dim='cases')['tas']
    gmstssp370=gmstsspt-base
    regtemp_ssp=transient_airtemps_regional
    return(gmstssp370,regtemp_ssp,gmst2,test,base_region,piCbase_region,base_mask,ts_hist_regional)
# gmstssp370[var],regtemp_ssp[var],gmst2[var],test[var],base_mask[var]=calc_regions(var)

/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_bq777.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_bu607.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_bw848.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_bw987.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_bz227.nc
/gws/nopw/j04/realproj/users/adittus/intermediate_files/stable/gmst/tas_gmst_cd269.nc
<xarray.DataArray 'year' (year: 500)>
array([2014, 2015, 2016, ..., 2511, 2512, 2513])
Coordinates:
  * year     (year) int64 2014 2015 2016 2017 2018 ... 2509 2510 2511 2512 2513
    height   float64 1.5


In [4]:
region='IPCC'
# print(regtemp_ssp[var])
for var in ['pr']:
    gmstssp370[var],regtemp_ssp[var],gmst2[var],test[var],base_region[var],piCbase_region[var],base_mask[var],hist[var]=calc_regions(var,units='%',mask='land')
#     # gmstssp370[var],regtemp_ssp[var],gmst2[var],test[var],base_region[var],piCbase_region[var],base_mask[var]=calc_regions(var,lat1=50,lat2=58,lon1=-11,lon2=2.5,units='%')
if (region != 'IPCC'):
    netcdf_dataset=xr.Dataset(
         data_vars=dict(
        pr_region_mean=(["run", "year"], test[var].values),
             # year=test[var].year,
             branch_year=[2014,2020,2025,2025,2030,2040],
             branch_scenario=['historical','SSP3-7.0','SSP3-7.0','SSP2-4.5','SSP1-1.9','SSP3-7.0']
         ),
        coords=dict(
            year=test[var].year,
        ),
        attrs=dict(region=reg,
                  season=seas,
                  branch_info=['hist 2014','2020 SSP370','2025 SSP370','2025 SSP245','2030 SSP119','2040 SSP370']),
    )
    netcdf_dataset.to_netcdf('timeseries/'+var+'_'+region+'_'+seas+'_stable.nc')
    netcdf_dataset=xr.Dataset(
         data_vars=dict(
        pr_region_mean=(["run", "year"], regtemp_ssp[var].values),
             # year=regtemp_ssp[var].year
    
         ),
        coords=dict(
            year=regtemp_ssp[var].year,
        ),
        attrs=dict(region=reg,
                  season=seas),
    )
    netcdf_dataset.to_netcdf('timeseries/'+var+'_'+region+'_'+seas+'_transient.nc')

else:
    netcdf_dataset=xr.Dataset(
         data_vars=dict(
        pr_region_mean=(["run", "year","region"], test[var].values),
             names=base_mask[var].names.values,
             # year=test[var].year,
             branch_year=[2014,2020,2025,2025,2030,2040],
             branch_scenario=['historical','SSP3-7.0','SSP3-7.0','SSP2-4.5','SSP1-1.9','SSP3-7.0']
         ),
        coords=dict(
            year=test[var].year,
        ),
        attrs=dict(region=reg,
                  season=seas,
                  branch_info=['hist 2014','2020 SSP370','2025 SSP370','2025 SSP245','2030 SSP119','2040 SSP370']),
    )
    netcdf_dataset.to_netcdf('timeseries/'+var+'_'+region+'_'+seas+'_stable_land.nc')
    
    netcdf_dataset=xr.Dataset(
         data_vars=dict(
        pr_region_mean=(["run", "year","region"], regtemp_ssp[var].values),
             # year=regtemp_ssp[var].year
    
         ),
        coords=dict(
            year=regtemp_ssp[var].year,
        ),
        attrs=dict(region=reg,
                  season=seas),
    )
    netcdf_dataset.to_netcdf('timeseries/'+var+'_'+region+'_'+seas+'_'+scenario+'_land.nc')
    
    netcdf_dataset=xr.Dataset(
         data_vars=dict(
        pr_region_mean=(["run", "year","region"], hist[var].values),
             # year=regtemp_ssp[var].year
    
         ),
        coords=dict(
            year=hist[var].year,
        ),
        attrs=dict(region=reg,
                  season=seas),
    )
    netcdf_dataset.to_netcdf('timeseries/'+var+'_'+region+'_'+seas+'_hist_land.nc')
        
        

<xarray.DataArray 'pr' (cases: 16, year: 165, lat: 144, lon: 192)>
dask.array<transpose, shape=(16, 165, 144, 192), dtype=float64, chunksize=(1, 1, 144, 192), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
  * year     (year) int64 1850 1851 1852 1853 1854 ... 2010 2011 2012 2013 2014
Dimensions without coordinates: cases
<xarray.DataArray (cases: 16, year: 165, lat: 144, lon: 192)>
dask.array<transpose, shape=(16, 165, 144, 192), dtype=float64, chunksize=(1, 1, 144, 192), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -89.38 -88.12 -86.88 -85.62 ... 86.88 88.12 89.38
  * lon      (lon) float64 0.9375 2.812 4.688 6.562 ... 353.4 355.3 357.2 359.1
    type     |S4 ...
  * year     (year) int64 1850 1851 1852 1853 1854 ... 2010 2011 2012 2013 2014
Dimensions without coordinates: cases
16.67890650057234
[[False False False ..